<a href="https://colab.research.google.com/github/Buck-61/cs290/blob/main/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("https://github.com/Buck-61/cs290/raw/refs/heads/main/heart.csv")

In [3]:
df.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [4]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df["output"].unique()

array([1, 0])

In [6]:
features = df.columns[ df.columns != "output" ]
features

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall'],
      dtype='object')

In [7]:
target = "output"
attribute = "age"

In [8]:
vals = df[ attribute].unique()
vals

array([63, 37, 41, 56, 57, 44, 52, 54, 48, 49, 64, 58, 50, 66, 43, 69, 59,
       42, 61, 40, 71, 51, 65, 53, 46, 45, 39, 47, 62, 34, 35, 29, 55, 60,
       67, 68, 74, 76, 70, 38, 77])

In [9]:
props = df[ df[attribute] == vals[8] ][target].value_counts( normalize=True )
props

,proportion
output,
1,0.571429
0,0.428571


In [10]:
entropy = 0
for p in props.array:
  entropy = entropy - p*math.log2(p)
entropy

0.9852281360342515

In [11]:
entropy = 0
for i in range(len(props)):
  entropy = entropy - props.iloc[i]*math.log2( props.iloc[i] )
entropy

0.9852281360342515

In [12]:
entropy = 0
overall = len( df )
for val in vals:
  subset_size = len(df[ df[attribute] == val ])
  weight = subset_size / overall
  props = df[ df[attribute] == val ][target].value_counts( normalize=True )
  for p in props.array:
    entropy =  entropy - weight*(p*math.log2(p))

entropy

0.8589593301514089

In [34]:
def Attribute_selection_method(df, target, choice):
  overall = len( df )
  vals = df[ attribute].unique()
  if choice == "entropy":
    entropy = 0
    for val in vals:
      subset_size = len(df[ df[attribute] == val ])
      weight = subset_size / overall
      props = df[ df[attribute] == val ][target].value_counts( normalize=True )
      for p in props.array:
        entropy =  entropy - weight*(p*math.log2(p))
    return entropy
  else:
    gini = 0
    for val in vals:
      subset_size = len(df[ df[attribute] == val ])
      weight = subset_size / overall
      props = df[ df[attribute] == val ][target].value_counts( normalize=True )
      gini = gini + weight*(1-sum(props**2))
    return gini

In [33]:
x = Attribute_selection_method(df, "output", "gini")
x

0.41396516612162026

In [44]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2)

In [45]:
train_set.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [46]:
train_set = train_set[["age", "sex", "cp", "trtbps", "chol", "thalachh", "exng"]]

In [47]:
test_set = test_set[["age", "sex", "cp", "trtbps", "chol", "thalachh", "exng"]]

X_train = train_set[["age", "cp", "trtbps", "chol", "thalachh", "exng"]]
y_train = train_set["output"]

In [48]:
x_test = train_set[["age", "cp", "trtbps", "chol", "thalachh", "exng"]]
y_test = train_set["sex"]

In [49]:
from sklearn.tree import DecisionTreeClassifier

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree